In [1]:
!pip install transformers

In [2]:
import json
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn


In [3]:
train_path = 'numerical_measuremental_conceptual_train.json'
dev_path = 'numerical_measuremental_conceptual_dev.json'
test_path = 'numerical_measuremental_conceptual_test.json'
train_json = open(train_path)
dev_json = open(dev_path)
test_json = open(test_path)
train_data = json.load(train_json)
dev_data = json.load(dev_json)
test_data = json.load(test_json)

In [4]:
from transformers import BertForMultipleChoice, BertTokenizer, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', use_fast = True)
#mathtokenizer = AutoTokenizer.from_pretrained('tbs17/MathBERT-custom', use_fast= True)
#modelmulti = BertForMultipleChoice.from_pretrained('bert-base-uncased')
#modelmultimath = BertForMultipleChoice.from_pretrained('tbs17/MathBERT-custom')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
def format_data(dataset):
  questions = []
  choices = []
  answers = []
  numerical = []
  measuremental = []
  conceptual = []
  total = 0
  have_all = 0
  o1 = 0
  o2 = 0
  for entry in dataset:
    total += 1
    if 'numerical_explanation' in entry.keys() and 'measuremental_explanation' in entry.keys() and 'conceptual_explanation' in entry.keys():
      numerical.append(entry['numerical_explanation'])
      measuremental.append(entry['measuremental_explanation'])
      conceptual.append(entry['conceptual_explanation'])
      questions.append(entry['question'])
      choices.append([entry['Option1'], entry['Option2']])
      if entry['answer'] == 'Option 1':
        answers.append(entry['Option1'])
        o1 += 1
      elif entry['answer'] == 'Option 2':
        answers.append(entry['Option2'])
        o2+= 1

      have_all += 1
  print(f'{have_all} entries used out of {total}. {(total-have_all)*100/total} of data was removed. {o1} entries had option 1 as an answer and {o2} entries had question 2 as an answer')
  return questions, choices, answers, numerical, measuremental, conceptual

In [ ]:
train_questions, train_choices, train_answers, train_numerical, train_measuremental, train_conceptual = format_data(train_data)
dev_questions, dev_choices, dev_answers, dev_numerical, dev_measuremental, dev_conceptual = format_data(dev_data)
test_questions, test_choices, test_answers, test_numerical, test_measuremental, test_conceptual = format_data(test_data)

In [7]:
def all_tokenizer(questions, choices, answers, tokenizer, numerical, measuremental, conceptual):
  one_choices = []
  one_questions = []
  numerical_explanations = []
  measuremental_explanations = []
  conceptual_explanations = []
  labels = []
  pairs = []
  for i in range(len(questions)):
    question = questions[i]
    if numerical != None:
      question = f'{question} {numerical[i]}'
    if measuremental != None:
      question = f'{question} {measuremental[i]}'
    if conceptual != None:
      question = f'{question} {conceptual[i]}'
    one_question = [question, question]
    one_choice = [f"{choice}" for choice in choices[i]]# input sequence: question ans1 question ans2

    one_questions.append(one_question)
    one_choices.append(one_choice)
    if i == 0:
      print(one_choice)
    labels.append(return_label(choices[i], answers[i]))#since 2 poss choices. if answer is o1, it's [0, 0] if o2 it's [1, 1]
    pair = tokenizer(one_question, one_choice, max_length=512, truncation=False, padding=True, return_tensors='pt')
    pairs.append(pair)
  keys = ['input_ids', 'attention_mask']
  print(pairs[0]['input_ids'].size())
  max_length = 512
  for ix, p in enumerate(pairs):
    assert len(p['input_ids']) == 2
    assert len(p['input_ids'][0]) == len(p['input_ids'][1])
    padding_length = max_length - len(p['input_ids'][0])
    if len(p['input_ids'][0]) >= 512:
      print(f'Question with index {ix}, with numerical_explanations {numerical}, measuremental_explanations {measuremental} and conceptual_explanations {conceptual} is at least 512 tokens long')
      raise Exception
    p['input_ids'] = torch.cat([p['input_ids'], torch.zeros((2, padding_length), dtype=torch.long)], dim=1)
    p['attention_mask'] = torch.cat([p['attention_mask'], torch.zeros((2, padding_length), dtype=torch.long)], dim=1)

  tokenized_examples = {}
  for k in keys:
    tokenized_examples[k] = torch.cat([p[k] for p in pairs], dim=0)
  num_samples = len(tokenized_examples['input_ids'])
  num_questions = num_samples//2
  print(num_samples/2, num_samples//2)
  print(len(questions))
  labels_tensor = torch.tensor(labels)
  print('labelstensor size', labels_tensor.size())
  print('tokenizedexamples inputids size', tokenized_examples['input_ids'].size())
  dictionary_context_to_questions = {
        'input_ids': tokenized_examples['input_ids'].view(num_questions, 2, -1),
        'attention_mask': tokenized_examples['attention_mask'].view(num_questions, 2, -1),
    }
  print('dictionarycontexttoquestions size', dictionary_context_to_questions['input_ids'].size())
  return dictionary_context_to_questions, labels_tensor

def return_label(choice_pair, correct_answer):
  if correct_answer == choice_pair[0]:
    return 0
  elif correct_answer == choice_pair[1]:
    return 1
  else:
    raise Exception


In [9]:
bu_train_preprocessed_b, bu_train_labels_b = all_tokenizer(train_questions, train_choices, train_answers, tokenizer, None, None, None)
bu_dev_preprocessed_b, bu_dev_labels_b = all_tokenizer(dev_questions, dev_choices, dev_answers, tokenizer, None, None, None)
bu_test_preprocessed_b, bu_test_labels_b = all_tokenizer(test_questions, test_choices, test_answers, tokenizer, None, None, None)

bu_train_preprocessed_n, bu_train_labels_n = all_tokenizer(train_questions, train_choices, train_answers, tokenizer, train_numerical, None, None)
bu_dev_preprocessed_n, bu_dev_labels_n = all_tokenizer(dev_questions, dev_choices, dev_answers, tokenizer, dev_numerical, None, None)
bu_test_preprocessed_n, bu_test_labels_n = all_tokenizer(test_questions, test_choices, test_answers, tokenizer, test_numerical, None, None)

bu_train_preprocessed_m, bu_train_labels_m = all_tokenizer(train_questions, train_choices, train_answers, tokenizer, None, train_measuremental, None)
bu_dev_preprocessed_m, bu_dev_labels_m = all_tokenizer(dev_questions, dev_choices, dev_answers, tokenizer, None, dev_measuremental, None)
bu_test_preprocessed_m, bu_test_labels_m = all_tokenizer(test_questions, test_choices, test_answers, tokenizer, None, test_measuremental, None)

bu_train_preprocessed_c, bu_train_labels_c = all_tokenizer(train_questions, train_choices, train_answers, tokenizer, None, None, train_conceptual)
bu_dev_preprocessed_c, bu_dev_labels_c = all_tokenizer(dev_questions, dev_choices, dev_answers, tokenizer, None, None, dev_conceptual)
bu_test_preprocessed_c, bu_test_labels_c = all_tokenizer(test_questions, test_choices, test_answers, tokenizer, None, None, test_conceptual)

bu_train_preprocessed_nm, bu_train_labels_nm = all_tokenizer(train_questions, train_choices, train_answers, tokenizer, train_numerical, train_measuremental, None)
bu_dev_preprocessed_nm, bu_dev_labels_nm = all_tokenizer(dev_questions, dev_choices, dev_answers, tokenizer, dev_numerical, dev_measuremental, None)
bu_test_preprocessed_nm, bu_test_labels_nm = all_tokenizer(test_questions, test_choices, test_answers, tokenizer, test_numerical, test_measuremental, None)

bu_train_preprocessed_nmc, bu_train_labels_nmc = all_tokenizer(train_questions, train_choices, train_answers, tokenizer, train_numerical, train_measuremental, train_conceptual)
bu_dev_preprocessed_nmc, bu_dev_labels_nmc = all_tokenizer(dev_questions, dev_choices, dev_answers, tokenizer, dev_numerical, dev_measuremental, dev_conceptual)
bu_test_preprocessed_nmc, bu_test_labels_nmc = all_tokenizer(test_questions, test_choices, test_answers, tokenizer, test_numerical, test_measuremental, test_conceptual)

['rottweiler', 'greyhound']


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2632: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


torch.Size([2, 40])
537.0 537
537
labelstensor size torch.Size([537])
tokenizedexamples inputids size torch.Size([1074, 512])
dictionarycontexttoquestions size torch.Size([537, 2, 512])
['the mountain was farther away', 'Jane was farther away']
torch.Size([2, 43])
67.0 67
67
labelstensor size torch.Size([67])
tokenizedexamples inputids size torch.Size([134, 512])
dictionarycontexttoquestions size torch.Size([67, 2, 512])
['Tina', 'Maria']
torch.Size([2, 42])
153.0 153
153
labelstensor size torch.Size([153])
tokenizedexamples inputids size torch.Size([306, 512])
dictionarycontexttoquestions size torch.Size([153, 2, 512])
['rottweiler', 'greyhound']
torch.Size([2, 46])
537.0 537
537
labelstensor size torch.Size([537])
tokenizedexamples inputids size torch.Size([1074, 512])
dictionarycontexttoquestions size torch.Size([537, 2, 512])
['the mountain was farther away', 'Jane was farther away']
torch.Size([2, 49])
67.0 67
67
labelstensor size torch.Size([67])
tokenizedexamples inputids size t

In [ ]:
'''train_preprocessed_b, train_labels_b = all_tokenizer(train_questions, train_choices, train_answers, mathtokenizer, None, None, None)
dev_preprocessed_b, dev_labels_b = all_tokenizer(dev_questions, dev_choices, dev_answers, mathtokenizer, None, None, None)
test_preprocessed_b, test_labels_b = all_tokenizer(test_questions, test_choices, test_answers, mathtokenizer, None, None, None)

train_preprocessed_n, train_labels_n = all_tokenizer(train_questions, train_choices, train_answers, mathtokenizer, train_numerical, None, None)
dev_preprocessed_n, dev_labels_n = all_tokenizer(dev_questions, dev_choices, dev_answers, mathtokenizer, dev_numerical, None, None)
test_preprocessed_n, test_labels_n = all_tokenizer(test_questions, test_choices, test_answers, mathtokenizer, test_numerical, None, None)

train_preprocessed_m, train_labels_m = all_tokenizer(train_questions, train_choices, train_answers, mathtokenizer, None, train_measuremental, None)
dev_preprocessed_m, dev_labels_m = all_tokenizer(dev_questions, dev_choices, dev_answers, mathtokenizer, None, dev_measuremental, None)
test_preprocessed_m, test_labels_m = all_tokenizer(test_questions, test_choices, test_answers, mathtokenizer, None, test_measuremental, None)

train_preprocessed_c, train_labels_c = all_tokenizer(train_questions, train_choices, train_answers, mathtokenizer, None, None, train_conceptual)
dev_preprocessed_c, dev_labels_c = all_tokenizer(dev_questions, dev_choices, dev_answers, mathtokenizer, None, None, dev_conceptual)
test_preprocessed_c, test_labels_c = all_tokenizer(test_questions, test_choices, test_answers, mathtokenizer, None, None, test_conceptual)

train_preprocessed_nm, train_labels_nm = all_tokenizer(train_questions, train_choices, train_answers, mathtokenizer, train_numerical, train_measuremental, None)
dev_preprocessed_nm, dev_labels_nm = all_tokenizer(dev_questions, dev_choices, dev_answers, mathtokenizer, dev_numerical, dev_measuremental, None)
test_preprocessed_nm, test_labels_nm = all_tokenizer(test_questions, test_choices, test_answers, mathtokenizer, test_numerical, test_measuremental, None)

train_preprocessed_nmc, train_labels_nmc = all_tokenizer(train_questions, train_choices, train_answers, mathtokenizer, train_numerical, train_measuremental, train_conceptual)
dev_preprocessed_nmc, dev_labels_nmc = all_tokenizer(dev_questions, dev_choices, dev_answers, mathtokenizer, dev_numerical, dev_measuremental, dev_conceptual)
test_preprocessed_nmc, test_labels_nmc = all_tokenizer(test_questions, test_choices, test_answers, mathtokenizer, test_numerical, test_measuremental, test_conceptual)'''


In [10]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertForMultipleChoice, get_linear_schedule_with_warmup
import nltk
nltk.download('punkt')
from nltk import sent_tokenize
import copy

class BertMultipleChoiceFineTuner:
    def __init__(self, model, model_name, lr, num_epochs=30, batch_size=8):
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.lr = lr

        if model == 'math':
            self.model = BertForMultipleChoice.from_pretrained('tbs17/MathBERT-custom')
        elif model == 'base':
            self.model = BertForMultipleChoice.from_pretrained('bert-base-uncased')
        else:
            raise Exception


        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.lr)

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        self.model_name = model_name






    def train(self, train_dictionary_context_to_questions, train_labels, dev_dictionary_context_to_questions, dev_labels):
        all_accuracies = []

        train_input_ids = train_dictionary_context_to_questions['input_ids']
        train_attention_masks = train_dictionary_context_to_questions['attention_mask']

        dev_input_ids = dev_dictionary_context_to_questions['input_ids']
        dev_attention_masks = dev_dictionary_context_to_questions['attention_mask']

        train_labels = train_labels.flatten()
        dev_labels = dev_labels.flatten()

        train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
        dev_dataset = TensorDataset(dev_input_ids, dev_attention_masks, dev_labels)

        train_dataloader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        dev_dataloader = DataLoader(dev_dataset, batch_size=self.batch_size)
        total_steps = len(train_dataloader) * self.num_epochs


        self.model.train()

        for epoch in range(self.num_epochs):
            total_train_loss = 0
            total_train_correct = 0

            for batch in train_dataloader:
                batch = tuple(t.to(self.device) for t in batch)
                input_ids, attention_mask, labels = batch

                self.optimizer.zero_grad()
                outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)#batch size is 8. despite for loop going through batches, the input_ids of all the batches are all entered in at once.
                loss = outputs.loss
                total_train_loss += loss.item()

                logits = outputs.logits
                _, predicted = torch.max(logits, 1)
                total_train_correct += (predicted == labels).sum().item()

                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 0.1)#parameters above the number are set to the num, prevents them from becoming too high

                self.optimizer.step()
            average_train_loss = total_train_loss / len(train_dataloader)
            accuracy = total_train_correct / len(train_dataset)
            model_state = {'epoch': epoch, 'model_state_dict': self.model.state_dict(), 'optimizer_state_dict': self.optimizer.state_dict()}
            model_filename = f'{self.model_name}_epoch_{epoch}_lr_{self.lr}.pt'
            torch.save(model_state, model_filename)
            del model_state
            # Validation

            self.model.eval()
            total_dev_loss = 0
            total_dev_correct = 0

            for batch in dev_dataloader:
                batch = tuple(t.to(self.device) for t in batch)
                input_ids, attention_mask, labels = batch

                with torch.no_grad():
                    outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                    loss = outputs.loss
                    total_dev_loss += loss.item()

                    logits = outputs.logits
                    _, predicted = torch.max(logits, 1)
                    total_dev_correct += (predicted == labels).sum().item()

            average_dev_loss = total_dev_loss / len(dev_dataloader)
            dev_accuracy = total_dev_correct / len(dev_dataset)
            all_accuracies.append(dev_accuracy)

            print(f'Epoch {epoch + 1}/{self.num_epochs} - Average Train Loss: {average_train_loss:.4f} with accuracy {accuracy} - Average Dev Loss: {average_dev_loss:.4f} with accuracy {dev_accuracy}')
        best = max(all_accuracies)
        best_epoch = all_accuracies.index(best)
        return best, best_epoch, self.lr


    def test(self, test_dictionary_context_to_questions, test_labels):
      test_input_ids = test_dictionary_context_to_questions['input_ids']#tokenized_train['input_ids']#ERROR: I'M TREATING TOKENIZED_TRAIN//TRAIN_INPUTS AS DICT WHEN IT'S A LIST
      test_attention_masks = test_dictionary_context_to_questions['attention_mask']#tokenized_train['attention_mask']

      test_labels = test_labels.flatten()
      test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)
      test_dataloader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False)
      self.model.eval()
      total_test_loss = 0
      total_test_correct = 0
      total_predictions = []
      total_labels = []
      total_input_ids = []

      for batch in test_dataloader:
          batch = tuple(t.to(self.device) for t in batch)
          input_ids, attention_mask, labels = batch

          with torch.no_grad():
              outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
              loss = outputs.loss
              print(type(outputs))
              total_test_loss += loss.item()

              logits = outputs.logits
              _, predicted = torch.max(logits, 1)
              total_input_ids.append(input_ids)
              total_predictions.append(predicted)
              total_labels.append(labels)
              print('increase in test correct', (predicted == labels).sum())
              total_test_correct += (predicted == labels).sum().item()

      average_test_loss = total_test_loss / len(test_dataloader)
      test_accuracy = total_test_correct / len(test_dataset)
      print(f'Loss on test set is {average_test_loss} and accuracy is {test_accuracy}')
      return total_predictions, total_input_ids, total_labels


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#goes through the learning rates, makes and trains models with each learning
#rate. For each model, deletes all the checkpoints except the one with the best
#accuracy. At the end notes which lr resulted in the best accuracy
#Based on part 4 of Dr Fredrik Wahlberg's code (https://github.com/fredrikwahlberg/5LN708/blob/master/Assignment%2001%20-%20Sentiment%20Polarity%20on%20Film%20Reviews.ipynb)
def compare_lrs(model_func, bert_model, name, train, train_labels, dev, dev_labels):
  lrs = [6e-6, 2e-6, 8e-7, 5e-7, 1e-7, 8e-8]#[6e-6, 4e-6, 2e-6, 8e-7, 6e-7, 4e-7, 2e-7, 8e-8]
  #lrs = lrs[3:]

  # Placeholder to make future comparissons easier
  best_lr = None
  print("Learning rate:\tTraining set accuracy\t Dev set accuracy:")
  num_epochs = 17

  for lr in lrs:
    print(lr)
    test_model = BertMultipleChoiceFineTuner(bert_model, name, lr=lr, num_epochs = num_epochs)
    best_accuracy, best_epoch, lrprinted = test_model.train(train, train_labels, dev, dev_labels)#train_questions, train_choices, train_answers, dev_questions, dev_choices, dev_answers)
    for ep in range(num_epochs):
      if ep != best_epoch:
        file_name = f'{name}_epoch_{ep}_lr_{lrprinted}.pt'
        print(file_name)
        !rm {file_name}
    with open('bestepochrecord.txt', 'a') as f3:
      print(f'Best epoch for lr {lrprinted} is {best_epoch} with dev accuracy {best_accuracy}', file=f3)
    print(f'Best epoch for lr {lrprinted} is {best_epoch} with dev accuracy {best_accuracy}')
    if test_model:
      del test_model

    if best_lr is None or best_lr[0] < best_accuracy:
      best_lr = (best_accuracy, best_epoch, lrprinted)

  print(f"Best model for {name} is the one with lr {best_lr[2]}, with dev accuracy {best_lr[0]} at epoch {best_lr[1]}")
  with open('bestmodelrecord.txt', 'a') as f2:
    print(f"Best model for {name} is the one with lr {best_lr[2]}, with dev accuracy {best_lr[0]} at epoch {best_lr[1]}", file=f2)


In [ ]:
import numpy as np
print('baseline_none_no_dev_explanations')
compare_lrs(BertMultipleChoiceFineTuner, 'base', 'baseline_none_no_dev_explanations', bu_train_preprocessed_b, bu_train_labels_b, bu_dev_preprocessed_b, bu_dev_labels_b)


In [ ]:
print('baseline_n_no_dev_explanations')
compare_lrs(BertMultipleChoiceFineTuner, 'base', 'baseline_n_no_dev_explanations', bu_train_preprocessed_n, bu_train_labels_n, bu_dev_preprocessed_b, bu_dev_labels_b)

In [ ]:
print('baseline_m_no_dev_explanations')
compare_lrs(BertMultipleChoiceFineTuner, 'base', 'baseline_m_no_dev_explanations', bu_train_preprocessed_m, bu_train_labels_m, bu_dev_preprocessed_b, bu_dev_labels_b)


In [ ]:
print('baseline_c_no_dev_explanations')
compare_lrs(BertMultipleChoiceFineTuner, 'base', 'baseline_c_no_dev_explanations', bu_train_preprocessed_c, bu_train_labels_c, bu_dev_preprocessed_b, bu_dev_labels_b)


In [ ]:
print('baseline_nm_no_dev_explanations')
compare_lrs(BertMultipleChoiceFineTuner, 'base', 'baseline_nm_no_dev_explanations', bu_train_preprocessed_nm, bu_train_labels_nm, bu_dev_preprocessed_b, bu_dev_labels_b)


In [ ]:
print('baseline_nmc_no_dev_explanations')
compare_lrs(BertMultipleChoiceFineTuner, 'base', 'baseline_nmc_no_dev_explanations', bu_train_preprocessed_nmc, bu_train_labels_nmc, bu_dev_preprocessed_b, bu_dev_labels_b)

In [ ]:
from google.colab import runtime
runtime.unassign()

In [14]:
#trains a model with the provided learning rate, deletes all checkpoints except the best one (to save space) and tests the data on the best epoch
def train_model(model_func, tokenizer, bert_model, name, train_data, train_labels, dev_data, dev_labels, lr, num_epochs=25):
  test_model = BertMultipleChoiceFineTuner(bert_model, name, lr=lr, num_epochs = num_epochs)
  best_accuracy, best_epoch, lrprinted = test_model.train(train_data, train_labels, dev_data, dev_labels)#train_questions, train_choices, train_answers, dev_questions, dev_choices, dev_answers)
  best_epoch = int(input(f'Currently, best_epoch is {best_epoch} starting from 0. Enter the epoch you want to use (reenter the best epoch to use that)'))
  return best_epoch, name, num_epochs, lrprinted, test_model

def test_model_func(best_epoch, name, num_epochs, lrprinted, test_model, test_data, test_labels):
  for ep in range(num_epochs):
    if ep != best_epoch:
      file_name = f'{name}_epoch_{ep}_lr_{lrprinted}.pt'
      print(file_name)
      !rm {file_name}
  with open('bestepochrecord.txt', 'a') as f3:
    print(f'Best epoch for {name} with lr {lrprinted} is {best_epoch}', file=f3)
  print(f'Best epoch for lr {lrprinted} is {best_epoch}')
  checkpoint = torch.load(f'{name}_epoch_{best_epoch}_lr_{lrprinted}.pt')
  test_model.model.load_state_dict(checkpoint['model_state_dict'])
  total_predictions, total_input_ids, total_labels = test_model.test(test_data, test_labels)
  extended_ids = []
  extended_labels = []
  extended_predictions = []
  answers = []
  for batch_list in total_input_ids:
    extended_ids.extend(batch_list)
  for batch_labels in total_labels:
    extended_labels.extend(batch_labels)
  for batch_answers in total_predictions:
    extended_predictions.extend(batch_answers)
  for i, entry in enumerate(extended_ids):
    first_q = entry[0]
    first_q = tokenizer.decode(first_q)
    first_q = first_q.split('[SEP]')
    question = first_q[0]
    choice_1 = first_q[1]
    second_q= entry[1]
    second_q = tokenizer.decode(second_q)
    second_q = second_q.split('[SEP]')
    choice_2 = second_q[1]
    prediction = extended_predictions[i].item()
    label = extended_labels[i].item()
    answer = question, choice_1, choice_2, prediction, label
    answers.append(answer)

  with open(f'{name}_result_data.txt', 'w') as f:
    for entr in answers:
      print(entr, file=f)
  if test_model:
      del test_model





In [15]:
import numpy as np
print('base_none_no_dev_explanations')
bu_b_epoch, bu_b_name, bu_b_num_epochs, bu_b_lrprinted, bu_b_model = train_model(BertMultipleChoiceFineTuner, tokenizer, 'base', 'base_none_no_dev_explanations', bu_train_preprocessed_b, bu_train_labels_b, bu_dev_preprocessed_b, bu_dev_labels_b, lr=6e-6)
test_model_func(bu_b_epoch, bu_b_name, bu_b_num_epochs, bu_b_lrprinted, bu_b_model, bu_test_preprocessed_b, bu_test_labels_b)


base_none_no_dev_explanations


Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/25 - Average Train Loss: 0.6973 with accuracy 0.5195530726256983 - Average Dev Loss: 0.6934 with accuracy 0.40298507462686567
Epoch 2/25 - Average Train Loss: 0.6912 with accuracy 0.5456238361266295 - Average Dev Loss: 0.6964 with accuracy 0.4925373134328358
Epoch 3/25 - Average Train Loss: 0.6752 with accuracy 0.62756052141527 - Average Dev Loss: 0.6964 with accuracy 0.5074626865671642
Epoch 4/25 - Average Train Loss: 0.6509 with accuracy 0.6536312849162011 - Average Dev Loss: 0.7087 with accuracy 0.4925373134328358
Epoch 5/25 - Average Train Loss: 0.6103 with accuracy 0.664804469273743 - Average Dev Loss: 0.7160 with accuracy 0.5074626865671642
Epoch 6/25 - Average Train Loss: 0.5993 with accuracy 0.6834264432029795 - Average Dev Loss: 0.7106 with accuracy 0.5223880597014925
Epoch 7/25 - Average Train Loss: 0.5830 with accuracy 0.702048417132216 - Average Dev Loss: 0.7337 with accuracy 0.44776119402985076
Epoch 8/25 - Average Train Loss: 0.5746 with accuracy 0.705772811918063

In [21]:
print('base_n_no_dev_explanations')
bu_n_epoch, bu_n_name, bu_n_num_epochs, bu_n_lrprinted, bu_n_model = train_model(BertMultipleChoiceFineTuner, tokenizer, 'base', 'base_n_no_dev_explanations', bu_train_preprocessed_n, bu_train_labels_n, bu_dev_preprocessed_b, bu_dev_labels_b, lr=6e-6)
test_model_func(bu_n_epoch, bu_n_name, bu_n_num_epochs, bu_n_lrprinted, bu_n_model, bu_test_preprocessed_b, bu_test_labels_b)



base_n_no_dev_explanations


Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/25 - Average Train Loss: 0.6964 with accuracy 0.5046554934823091 - Average Dev Loss: 0.6908 with accuracy 0.5223880597014925
Epoch 2/25 - Average Train Loss: 0.6913 with accuracy 0.5176908752327747 - Average Dev Loss: 0.6921 with accuracy 0.4925373134328358
Epoch 3/25 - Average Train Loss: 0.6638 with accuracy 0.6350093109869647 - Average Dev Loss: 0.6978 with accuracy 0.5522388059701493
Epoch 4/25 - Average Train Loss: 0.6349 with accuracy 0.659217877094972 - Average Dev Loss: 0.6952 with accuracy 0.5074626865671642
Epoch 5/25 - Average Train Loss: 0.6038 with accuracy 0.7057728119180633 - Average Dev Loss: 0.7048 with accuracy 0.5373134328358209
Epoch 6/25 - Average Train Loss: 0.5540 with accuracy 0.7355679702048417 - Average Dev Loss: 0.7156 with accuracy 0.5373134328358209
Epoch 7/25 - Average Train Loss: 0.5057 with accuracy 0.7523277467411545 - Average Dev Loss: 0.7440 with accuracy 0.5373134328358209
Epoch 8/25 - Average Train Loss: 0.4582 with accuracy 0.77281191806331

In [17]:
print('base_m_no_dev_explanations')
bu_m_epoch, bu_m_name, bu_m_num_epochs, bu_m_lrprinted, bu_m_model = train_model(BertMultipleChoiceFineTuner, tokenizer, 'base', 'base_m_no_dev_explanations', bu_train_preprocessed_m, bu_train_labels_m, bu_dev_preprocessed_b, bu_dev_labels_b, lr=8e-7)
test_model_func(bu_m_epoch, bu_m_name, bu_m_num_epochs, bu_m_lrprinted, bu_m_model, bu_test_preprocessed_b, bu_test_labels_b)


base_m_no_dev_explanations


Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/25 - Average Train Loss: 0.6986 with accuracy 0.4860335195530726 - Average Dev Loss: 0.6919 with accuracy 0.5223880597014925
Epoch 2/25 - Average Train Loss: 0.6931 with accuracy 0.5139664804469274 - Average Dev Loss: 0.6922 with accuracy 0.4925373134328358
Epoch 3/25 - Average Train Loss: 0.6913 with accuracy 0.5754189944134078 - Average Dev Loss: 0.6924 with accuracy 0.47761194029850745
Epoch 4/25 - Average Train Loss: 0.6886 with accuracy 0.6350093109869647 - Average Dev Loss: 0.6927 with accuracy 0.4626865671641791
Epoch 5/25 - Average Train Loss: 0.6855 with accuracy 0.6350093109869647 - Average Dev Loss: 0.6929 with accuracy 0.4626865671641791
Epoch 6/25 - Average Train Loss: 0.6790 with accuracy 0.6424581005586593 - Average Dev Loss: 0.6933 with accuracy 0.4626865671641791
Epoch 7/25 - Average Train Loss: 0.6716 with accuracy 0.6629422718808193 - Average Dev Loss: 0.6937 with accuracy 0.47761194029850745
Epoch 8/25 - Average Train Loss: 0.6639 with accuracy 0.68528864059

In [22]:

print('base_c_no_dev_explanations')
bu_c_epoch, bu_c_name, bu_c_num_epochs, bu_c_lrprinted, bu_c_model = train_model(BertMultipleChoiceFineTuner, tokenizer, 'base', 'base_c_no_dev_explanations', bu_train_preprocessed_c, bu_train_labels_c, bu_dev_preprocessed_b, bu_dev_labels_b, lr=5e-7)
test_model_func(bu_c_epoch, bu_c_name, bu_c_num_epochs, bu_c_lrprinted, bu_c_model, bu_test_preprocessed_b, bu_test_labels_b)


base_c_no_dev_explanations


Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/25 - Average Train Loss: 0.6939 with accuracy 0.5065176908752328 - Average Dev Loss: 0.6960 with accuracy 0.4925373134328358
Epoch 2/25 - Average Train Loss: 0.6937 with accuracy 0.478584729981378 - Average Dev Loss: 0.6962 with accuracy 0.4626865671641791
Epoch 3/25 - Average Train Loss: 0.6914 with accuracy 0.527001862197393 - Average Dev Loss: 0.6969 with accuracy 0.43283582089552236
Epoch 4/25 - Average Train Loss: 0.6867 with accuracy 0.5716945996275605 - Average Dev Loss: 0.6977 with accuracy 0.44776119402985076
Epoch 5/25 - Average Train Loss: 0.6820 with accuracy 0.5810055865921788 - Average Dev Loss: 0.6994 with accuracy 0.43283582089552236
Epoch 6/25 - Average Train Loss: 0.6775 with accuracy 0.5921787709497207 - Average Dev Loss: 0.7006 with accuracy 0.5074626865671642
Epoch 7/25 - Average Train Loss: 0.6743 with accuracy 0.590316573556797 - Average Dev Loss: 0.7008 with accuracy 0.4925373134328358
Epoch 8/25 - Average Train Loss: 0.6753 with accuracy 0.5959031657355

In [19]:

print('base_nm_no_dev_explanations')
bu_nm_epoch, bu_nm_name, bu_nm_num_epochs, bu_nm_lrprinted, bu_nm_model = train_model(BertMultipleChoiceFineTuner, tokenizer, 'base', 'base_nm_no_dev_explanations', bu_train_preprocessed_nm, bu_train_labels_nm, bu_dev_preprocessed_b, bu_dev_labels_b, lr=5e-7)
test_model_func(bu_nm_epoch, bu_nm_name, bu_nm_num_epochs, bu_nm_lrprinted, bu_nm_model, bu_test_preprocessed_b, bu_test_labels_b)


base_nm_no_dev_explanations


Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/25 - Average Train Loss: 0.6984 with accuracy 0.478584729981378 - Average Dev Loss: 0.6925 with accuracy 0.4626865671641791
Epoch 2/25 - Average Train Loss: 0.6930 with accuracy 0.5009310986964618 - Average Dev Loss: 0.6924 with accuracy 0.417910447761194
Epoch 3/25 - Average Train Loss: 0.6919 with accuracy 0.5586592178770949 - Average Dev Loss: 0.6925 with accuracy 0.4925373134328358
Epoch 4/25 - Average Train Loss: 0.6907 with accuracy 0.5921787709497207 - Average Dev Loss: 0.6924 with accuracy 0.5223880597014925
Epoch 5/25 - Average Train Loss: 0.6893 with accuracy 0.6014897579143389 - Average Dev Loss: 0.6923 with accuracy 0.5074626865671642
Epoch 6/25 - Average Train Loss: 0.6864 with accuracy 0.6014897579143389 - Average Dev Loss: 0.6923 with accuracy 0.4925373134328358
Epoch 7/25 - Average Train Loss: 0.6822 with accuracy 0.5865921787709497 - Average Dev Loss: 0.6916 with accuracy 0.5074626865671642
Epoch 8/25 - Average Train Loss: 0.6771 with accuracy 0.594040968342644

KeyboardInterrupt: Interrupted by user

In [20]:

print('base_nmc_no_dev_explanations')
bu_nmc_epoch, bu_nmc_name, bu_nmc_num_epochs, bu_nmc_lrprinted, bu_nmc_model = train_model(BertMultipleChoiceFineTuner, tokenizer, 'base', 'base_nmc_no_dev_explanations', bu_train_preprocessed_nmc, bu_train_labels_nmc, bu_dev_preprocessed_b, bu_dev_labels_b, lr=5e-7)
test_model_func(bu_nmc_epoch, bu_nmc_name, bu_nmc_num_epochs, bu_nmc_lrprinted, bu_nmc_model, bu_test_preprocessed_b, bu_test_labels_b)

base_nmc_no_dev_explanations


Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/25 - Average Train Loss: 0.6970 with accuracy 0.48417132216014896 - Average Dev Loss: 0.6939 with accuracy 0.47761194029850745
Epoch 2/25 - Average Train Loss: 0.6934 with accuracy 0.48417132216014896 - Average Dev Loss: 0.6939 with accuracy 0.4626865671641791
Epoch 3/25 - Average Train Loss: 0.6919 with accuracy 0.5381750465549349 - Average Dev Loss: 0.6940 with accuracy 0.44776119402985076
Epoch 4/25 - Average Train Loss: 0.6899 with accuracy 0.5772811918063314 - Average Dev Loss: 0.6941 with accuracy 0.4626865671641791
Epoch 5/25 - Average Train Loss: 0.6882 with accuracy 0.5828677839851024 - Average Dev Loss: 0.6943 with accuracy 0.47761194029850745
Epoch 6/25 - Average Train Loss: 0.6848 with accuracy 0.595903165735568 - Average Dev Loss: 0.6946 with accuracy 0.47761194029850745
Epoch 7/25 - Average Train Loss: 0.6797 with accuracy 0.6052141527001862 - Average Dev Loss: 0.6948 with accuracy 0.47761194029850745
Epoch 8/25 - Average Train Loss: 0.6749 with accuracy 0.6089385